In [2]:
#from keras.applications.vgg16 import VGG16
#from keras.preprocessing import image
#from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

Using TensorFlow backend.


Racket and other things from amazon

In [3]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
import keras

In [ ]:
#import ssl

#try:
 #   _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
 #   pass
#else:
    # Handle target environment that doesn't support HTTPS verification
 #   ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
#model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
#model_vgg16_conv.summary()

In [ ]:
#pre-processing
#get data
trainLocation="./DogBreed/train/"
t=os.listdir(trainLocation)

In [ ]:
datatrain=[trainLocation+j for j in t]

In [ ]:
datatrainImages=[cv2.imread(j,0) for j in datatrain]

In [ ]:
m=[a.shape for a in datatrainImages]

In [ ]:
resizedimages=[cv2.resize(img,None,fx=256/max(img.shape),fy= 256/max(img.shape), interpolation = cv2.INTER_AREA) for img in  datatrainImages]

In [ ]:
moddatatrainImages=[np.pad(a, [(0, 256-a.shape[0]), (0, 256-a.shape[1])], mode='constant') for a in resizedimages]

In [ ]:
moddatatrainImages=np.array(moddatatrainImages)

In [ ]:
del(m);del(resizedimages);del(datatrain);del(datatrainImages)

In [ ]:
#plt.imshow(histmodtrainimages[1])
#plt.show()

In [ ]:
plt.imshow(moddatatrainImages[1])
plt.show()

In [ ]:
#histmodtrainimages=[cv2.equalizeHist(img) for img in moddatatrainImages];del(histmodtrainimages)

In [ ]:
moddatatrainImagesmean=127.5;#np.mean(moddatatrainImages.flatten());
moddatatrainImagesstd=86.0;#np.std(moddatatrainImages.flatten());

In [ ]:
moddatatrainImages=(moddatatrainImages-moddatatrainImagesmean)/moddatatrainImagesstd

In [ ]:
np.save("trainimagescnn",moddatatrainImages)

In [36]:
np.save("y_y",y_y)

In [4]:
labelstrain=pd.read_csv('./DogBreed/labels.csv');
labelstrain=labelstrain['breed'];
dlabs=set(labelstrain);
diclabs={a[1]:a[0] for a in enumerate(dlabs)};
x_x=np.load('trainimagescnn.npy');
ohcc=np.eye(120)
y_y=np.array([ohcc[diclabs[j]] for j in labelstrain])
x_train=x_x[:7000]; y_train=y_y[:7000];x_test=x_x[7000:];y_test=y_y[7000:]

In [5]:
x_train.shape

(7000, 256, 256)

In [6]:
y_train.shape

(7000, 120)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [8]:
from keras.layers.normalization import BatchNormalization
import time

In [32]:
model = Sequential()
batch_size=1024*4
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(16, (3, 3), batch_input_shape=(None,256, 256,1),padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3),padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3),padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3),padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(32, (3, 3),padding='same',use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

#model.add(Conv2D(64, (3, 3),padding='same',use_bias=False))
#model.add(BatchNormalization())
#model.add(Activation('relu'))

#model.add(MaxPooling2D(pool_size=(3, 3)))

#model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
#model.add(Conv2D(16, (3, 3), activation='relu',padding='same',use_bias=True))
#model.add(MaxPooling2D(pool_size=(4, 4)))
#model.add(Dropout(0.25))
#model.add(Dropout(0.25))
#model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
#model.add(Conv2D(16, (3, 3), activation='relu',padding='same',use_bias=True))
#model.add(MaxPooling2D(pool_size=(4, 4)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(120,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 256, 256, 16)      144       
_________________________________________________________________
batch_normalization_51 (Batc (None, 256, 256, 16)      64        
_________________________________________________________________
activation_51 (Activation)   (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 256, 256, 32)      4608      
_________________________________________________________________
batch_normalization_52 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
activation_52 (Activation)   (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 51, 51, 32)        0         
__________

In [34]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.1),#'sgd',
              metrics=['accuracy'])

In [35]:
model.fit(np.transpose(np.array([x_train]),(1,2,3,0)), y_train, epochs=200, batch_size=32)

Epoch 1/200
7000/7000 [==============================] - 2454s - loss: 4.9325 - acc: 0.0157  
Epoch 2/200
5696/7000 [=======================>......] - ETA: 210s - loss: 4.5293 - acc: 0.0392

KeyboardInterrupt: 